In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import query, Search
from pprint import pprint
from IPython.display import display, JSON 

In [2]:
ELASTIC_URL = "http://search.sirene.dataeng.etalab.studio:80"
ELASTIC_USER = "elastic"
ELASTIC_PASSWORD = "etalab123"

In [3]:
client = Elasticsearch(
    ELASTIC_URL,
    http_auth=(ELASTIC_USER, ELASTIC_PASSWORD)
)

In [4]:
client.info()

{'name': 'elastic-sirene',
 'cluster_name': 'es-docker-cluster',
 'cluster_uuid': 'UU-S6-DJSl2qi4YOAneMPQ',
 'version': {'number': '7.12.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '3186837139b9c6b6d23c3200870651f10d3343b7',
  'build_date': '2021-04-20T20:56:39.040728659Z',
  'build_snapshot': False,
  'lucene_version': '8.8.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [5]:
s = Search(using=client, index="siren-green")

In [6]:
"""
should means: If these clauses match, they increase the _score;
otherwise, they have no effect.
They are simply used to refine the relevance score for each document.
"""


'\nshould means: If these clauses match, they increase the _score;\notherwise, they have no effect.\nThey are simply used to refine the relevance score for each document.\n'

In [7]:
terms = "la poste paris"

In [8]:
 {
          "function_score": {
            "query": {
              "multi_match": {
                "query": "term",
                "fields": [
                  "nom_complet^3",
                  "etablissements.adresse",
                ],
                "type": "cross_fields",
                "operator": "AND",
              }
            },
            "field_value_factor": {
              "field": "nombre_etablissements_ouverts",
              "factor": 1,
              "modifier": "log1p",
              "missing": 1
            },
          }
        },

({'function_score': {'query': {'multi_match': {'query': 'term',
     'fields': ['nom_complet^3', 'etablissements.adresse'],
     'type': 'cross_fields',
     'operator': 'AND'}},
   'field_value_factor': {'field': 'nombre_etablissements_ouverts',
    'factor': 1,
    'modifier': 'log1p',
    'missing': 1}}},)

In [95]:
query_body = {
    "bool": {
      "should": [
          {
          "function_score": {
            "query": {
              "match_phrase":{ 
                "identifiant_association_unite_legale":
                {
                "query": terms,
                "boost": 50,
                "_name": "exact match identifiant association"
              }
            }},
            "field_value_factor": {
              "field": "nombre_etablissements_ouverts",
              "factor": 1,
              "modifier": "log1p",
              "missing": 1
            },
          }
        },
        {
          "function_score": {
            "query": {
              "match_phrase":{ 
                "nom_complet.keyword":
                {
                "query": terms,
                "boost": 100,
                "_name": "exact nom_complet match"
              }
            }},
            "field_value_factor": {
              "field": "nombre_etablissements_ouverts",
              "factor": 1,
              "modifier": "log1p",
              "missing": 1
            },
          }
        },
          {
          "function_score": {
            "query": {
              "match":{ 
                "nom_complet":
                {
                "query": terms,
                "operator": "AND",
                "boost": 50,
                "_name": "partial nom_complet match with AND"
              }
            }},
            "field_value_factor": {
              "field": "nombre_etablissements_ouverts",
              "factor": 1,
              "modifier": "log1p",
              "missing": 1
            },
          }
        },
          
        {
          "function_score": {
            "query": {
              "nested": {
                "path": "etablissements",
                "query": {
                  "bool": {
                    "should": [
                        { 
                        "match_phrase": {
                            "etablissements.enseigne_1.keyword": 
                            { 
                                "query": terms,
                                "boost": 25,
                                "_name": "exact match enseigne 1"
                            }
                        }
                            },
                        {
                         "match": {
                            "etablissements.enseigne_1": 
                            { 
                                "query": terms,
                                "operator": "AND",
                                "boost": 1,
                                "_name": "partial match enseigne 1"
                            }
                        }
                        },
                        { 
                        "match_phrase": {
                            "etablissements.enseigne_2.keyword": 
                            { 
                                "query": terms,
                                "boost": 25,
                                "_name": "exact match enseigne 2"
                            }
                        }
                            },
                        {
                         "match": {
                            "etablissements.enseigne_2": 
                            { 
                                "query": terms,
                                "operator": "AND",
                                "boost": 1,
                                "_name": "partial match enseigne 2"
                            }
                        }
                        },
                        { 
                        "match_phrase": {
                            "etablissements.enseigne_3.keyword": 
                            { 
                                "query": terms,
                                "boost": 25,
                                "_name": "exact match enseigne 3"
                            }
                        }
                            },
                        {
                         "match": {
                            "etablissements.enseigne_3": 
                            { 
                                "query": terms,
                                "operator": "AND",
                                "boost": 1,
                                "_name": "partial match enseigne 3"
                            }
                        }
                        },
                        {
                         "match": {
                            "etablissements.adresse": 
                            { 
                                "query": terms,
                                "operator": "AND",
                                "boost": 15,
                                "_name": "partial match adresse"
                            }
                        }
                        },
                        {
                         "match": {
                            "etablissements.geo_adresse": 
                            { 
                                "query": terms,
                                "operator": "AND",
                                "boost": 5,
                                "_name": "partial match adresse"
                            }
                        }
                        },
                        {
                        "multi_match": {
                            "query": terms,
                            "fields": [
                                "etablissements.nom_complet^50",
                                "etablissements.adresse",
                                "etablissements.concat_enseigne_adresse_siren_siret",
                            ],
                            "type": "cross_fields",
                            "operator": "AND",
                            "boost": 5,
                            "_name": "match nom complet et adresse",
                        }
                        },
                         
                    ]
                  }
                },
                  "inner_hits": {}
              }},
            
          "field_value_factor": {
              "field": "nombre_etablissements_ouverts",
              "factor": 1,
              "modifier": "log1p",
              "missing": 1
            },
          }
        },
         {
          "function_score": {
            "query": {
              "match":{ 
                "liste_dirigeants":
                {
                "query": terms,
                "operator": "AND",
                "boost": 10,
                "_name": "partial match liste dirigeants"
              }
            }},
            "field_value_factor": {
              "field": "nombre_etablissements_ouverts",
              "factor": 1,
              "modifier": "log1p",
              "missing": 1
            },
          }
        },
        {
          "function_score": {
            "query": {
              "match":{ 
                "liste_elus":
                {
                "query": terms,
                "operator": "AND",
                "boost": 10,
                "_name": "partial match liste élus"
              }
            }},
            "field_value_factor": {
              "field": "nombre_etablissements_ouverts",
              "factor": 1,
              "modifier": "log1p",
              "missing": 1
            },
          }
        },
        
      ],
    }
  }



In [ ]:
{
          "function_score": {
              
            "query":{
                        "multi_match": {
                            "query": terms,
                            "fields": [
                                "nom_complet^10",
                                "liste_dirigeants^5",
                                "liste_élus^5",
                                "siren",
                            ],
                            "type": "best_fields",
                            "_name": "match nom complet, dirigeants, élus, siren",
                        }
                        },
            "field_value_factor": {
              "field": "nombre_etablissements_ouverts",
              "factor": 1,
              "modifier": "log1p",
              "missing": 1
            },
          }
        },
       

In [94]:
terms = "ganymede"

In [96]:
response = client.search(index="siren-green", query=query_body, explain=True)

In [97]:
response['hits']['total']['value']

83

In [98]:
display(JSON(response))

<IPython.core.display.JSON object>

In [18]:
# expanded=True
for hit in response['hits']['hits']:
    display(JSON(hit["_source"]))

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

In [52]:
# expanded=True
for hit in response['hits']['hits']:
    display(JSON(hit["inner_hits"]))

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>